# Import Libaries

In [78]:
import numpy as np
import pandas as pd
import pickle
import os
import shutil

## Create directory management

In [75]:
# If dir not present creatre
pickel_dir = "saved_model"
if not os.path.exists(pickel_dir):
    os.makedirs(pickel_dir)

## Funnel feature

In [59]:
dataset = pd.read_csv('slider_all.csv.xz')
dataset = dataset[['is_normal', 'T_rms_mean', 'T_rms_std', 'T_zcr_mean',
                       'F_mel_mean', 'F_mel_std', 'F_mel_rms_mean', 'F_mel_rms_std',
                       'F_mfcc_mean', 'F_mfcc_std', 'F_flatness_mean',
                       'F_bandwidth_mean', 'F_bandwidth_std', 'F_contrast_mean',
                       'F_rolloff_mean', 'F_rolloff_std']]

## Split data

In [60]:
X = dataset.drop("is_normal", axis=1)
y = dataset["is_normal"]

from sklearn.model_selection import train_test_split

X_model, X_val, y_model, y_val  = train_test_split(X, y, test_size=0.1, random_state=1)

## Resample over sample

In [61]:
from imblearn.over_sampling import BorderlineSMOTE
X_resample, y_resample = BorderlineSMOTE().fit_resample(X_model, y_model.ravel())

y_resample = pd.DataFrame(y_resample)
X_resample = pd.DataFrame(X_resample)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size=0.2, random_state=1)

## Random Forest Estimator 100

In [62]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [63]:
from sklearn.ensemble import RandomForestClassifier

no_estimators = 100

RF_classifier = RandomForestClassifier(random_state = 0, n_estimators = no_estimators, criterion = 'entropy')
RF_classifier.fit(X_train, y_train.values.ravel())

RandomForestClassifier(criterion='entropy', random_state=0)

In [64]:
y_pred = RF_classifier.predict(X_test)

from sklearn.metrics import classification_report

target_names = []
for i in y.unique():
    target_names.append(f"class {i}")
y_true = y_test[0].tolist()
y_pred = y_pred.tolist()
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.94      0.97      0.95      1736
     class 1       0.97      0.94      0.95      1725

    accuracy                           0.95      3461
   macro avg       0.95      0.95      0.95      3461
weighted avg       0.95      0.95      0.95      3461



In [65]:
y_valid_pred = RF_classifier.predict(X_val)

from sklearn.metrics import classification_report

target_names = []
for i in y.unique():
    target_names.append(f"class {i}")
y_true = y_val.tolist()
y_pred = y_valid_pred.tolist()
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.77      0.91      0.83       268
     class 1       0.97      0.92      0.95       961

    accuracy                           0.92      1229
   macro avg       0.87      0.92      0.89      1229
weighted avg       0.93      0.92      0.92      1229



In [81]:
filename = 'Predict_slider.sav'
pickle.dump(RF_classifier, open(f"./{pickel_dir}/{filename}", 'wb'))

# load the model from disk
loaded_model = pickle.load(open(f"./{pickel_dir}/{filename}", 'rb'))
result = loaded_model.score(X_val, y_val)
print(result)

0.9210740439381611
